# Content-Based Movie Recommendation

Dataset Source:

https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots/

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv("../data/wiki_movie_plots_deduped.csv")
df.shape

(34886, 8)

In [4]:
df.sample(5)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
21275,2012,The Expatriate,British,Director: Philipp Stölzl,Director: Philipp Stölzl\r\nCast: Aaron Eckhar...,unknown,https://en.wikipedia.org/wiki/The_Expatriate,Ben Logan is an American single parent who has...
9996,1982,An Officer and a Gentleman,American,Taylor Hackford,"Richard Gere, Louis Gossett, Jr., Debra Winger...",drama,https://en.wikipedia.org/wiki/An_Officer_and_a...,"Zachary ""Zack"" Mayo is preparing to report to ..."
7728,1963,Goldilocks and the Three Bares,American,Herschell Gordon Lewis,"Alison Edwards, Rex Marlow, Thomas Sweetwood, ...",erotic comedy,https://en.wikipedia.org/wiki/Goldilocks_and_t...,The film follows the perils of nightclub singe...
33674,2012,Kaizoku Sentai Gokaiger vs. Space Sheriff Gava...,Japanese,Shojiro Nakazawa,"Ryota Ozawa, Yuki Yamada, Mao Ichimichi",tokusatsu,https://en.wikipedia.org/wiki/Kaizoku_Sentai_G...,The film begins with the Gokai Galleon being c...
32451,2011,Golconda High School,Telugu,Mohan Krishna Indraganti,"Sumanth, Swati Reddy",drama,https://en.wikipedia.org/wiki/Golconda_High_Sc...,Golconda High School's Board’s Trustees' membe...


## Converting to String

In [5]:
df.columns

Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot'],
      dtype='object')

In [6]:
df = df.astype(str)
df.dtypes

Release Year        object
Title               object
Origin/Ethnicity    object
Director            object
Cast                object
Genre               object
Wiki Page           object
Plot                object
dtype: object

## Data Cleaning

Action Items:

* Lower-Case the whole data frame
* Director: Removing 'Director:' and 'Cast:'
* Director, Cast: Removing '\r\n', '\n' and '\r'

* Genre: Replacing '/' with Space
* Director, Cast, Genre: Removing 'Uknonwn' and 'Nan'

* Director: Separating Directors and Actors names
* Director, Cast: Checking if the names are separated with ' and ', ' & '

* Director, Cast: Merging the first names and last names together
* Director, Cast: Adding the words of 'Director' and 'Actor' as prefix

* Plot: Removing English Stopwords
* Doc: Removing special characters

In [7]:
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

<ipython-input-7-614eaff662ae>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [8]:
df["Director"] = df["Director"].str.replace("director:", "", regex=False)
df["Director"] = df["Director"].str.replace("cast:", "", regex=False)

df["Director"] = df["Director"].str.replace("\r\n", " ", regex=False)
df["Cast"] = df["Cast"].str.replace("\r\n", " ", regex=False)

df["Director"] = df["Director"].str.replace("\n", " ", regex=False)
df["Cast"] = df["Cast"].str.replace("\n", " ", regex=False)

df["Genre"] = df["Genre"].str.replace("/", " ", regex=False)

df["Director"] = df["Director"].str.replace("unknown", "", regex=False)
df["Cast"] = df["Cast"].str.replace("unknown", "", regex=False)
df["Genre"] = df["Genre"].str.replace("unknown", "", regex=False)

df["Director"] = df["Director"].str.replace("nan", "", regex=False)
df["Cast"] = df["Cast"].str.replace("nan", "", regex=False)
df["Genre"] = df["Genre"].str.replace("nan", "", regex=False)


df["Director"] = df["Director"].str.replace(" and ", ",", regex=False)
df["Cast"] = df["Cast"].str.replace(" and ", ",", regex=False)
df["Director"] = df["Director"].str.replace(" & ", ",", regex=False)
df["Cast"] = df["Cast"].str.replace(" & ", ",", regex=False)


df["Director"] = df["Director"].str.replace(" ", "", regex=False)
df["Cast"] = df["Cast"].str.replace(" ", "", regex=False)

df["Director"] = df["Director"].str.replace(",", " ", regex=False)
df["Cast"] = df["Cast"].str.replace(",", " ", regex=False)


In [9]:
df["Director"] = np.where(df["Director"].str.len() > 0,
                          'director' + df["Director"],
                          df["Director"])

df["Cast"] = np.where(df["Cast"].str.len() > 0,
                      'actor' + df["Cast"],
                      df["Cast"])

df["Director"] = df["Director"].str.replace(" ", " director", regex=False)
df["Cast"] = df["Cast"].str.replace(" ", " actor", regex=False)


In [10]:
df.sample(5)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
15925,2010,salt,american,directorphillipnoyce,actorangelinajolie actorlievschreiber actorchi...,action thriller,https://en.wikipedia.org/wiki/salt_(2010_film),evelyn salt (angelina jolie) is being tortured...
31076,2011,deiva thirumagal,tamil,directora.l.vijay,actorvikram actoranushkashetty actoramalapaul ...,drama,https://en.wikipedia.org/wiki/deiva_thirumagal,krishna (vikram) is intellectually disabled an...
25517,1989,daata,bollywood,directorsultanahmed,actormithunchakravorty.shammikapoor actorpadmi...,,https://en.wikipedia.org/wiki/daata,dinanath is a school-teacher in a small town i...
12871,1996,twister,american,directorjandebont,actorhelenhunt actorbillpaxton actorjamigertz ...,disaster,https://en.wikipedia.org/wiki/twister_(1996_film),"one stormy night in june 1969, a monstrous f5 ..."
1710,1934,in old santa fe,american,directordavidhoward,actorkenmaynard actorevalynknapp,western,https://en.wikipedia.org/wiki/in_old_santa_fe,a cowboy named kentucky ken (ken maynard) and ...


## Merging the document

In [19]:
column_weights = {"Release Year": 10,
                  "Title": 1,
                  "Origin/Ethnicity": 5,
                  "Director": 5,
                  "Cast": 1,
                  "Genre": 10,
                  "Plot": 1}

df["doc"] = ""

for col in column_weights.keys():
    df["doc"] += column_weights[col] * (df[col] + ' ')

df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,doc
0,1901,kansas saloon smashers,american,,,,https://en.wikipedia.org/wiki/kansas_saloon_smashers,"a bartender is working at a saloon, serving drinks to customers. after he fills a stereotypically irish man's bucket with beer, carrie nation and her followers burst inside. they assault the irish man, pulling his hat over his eyes and then dumping the beer over his head. the group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. the bartender then sprays seltzer water in nation's face before a group of policemen appear and order everybody to leave.[1]","1901 1901 1901 1901 1901 1901 1901 1901 1901 1901 kansas saloon smashers american american american american american a bartender is working at a saloon, serving drinks to customers. after he fills a stereotypically irish man's bucket with beer, carrie nation and her followers burst inside. they assault the irish man, pulling his hat over his eyes and then dumping the beer over his head. the group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. the bartender then sprays seltzer water in nation's face before a group of policemen appear and order everybody to leave.[1]"
1,1901,love by the light of the moon,american,,,,https://en.wikipedia.org/wiki/love_by_the_light_of_the_moon,"the moon, painted with a smiling face hangs over a park at night. a young couple walking past a fence learn on a railing and look up. the moon smiles. they embrace, and the moon's smile gets bigger. they then sit down on a bench by a tree. the moon's view is blocked, causing him to frown. in the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better.","1901 1901 1901 1901 1901 1901 1901 1901 1901 1901 love by the light of the moon american american american american american the moon, painted with a smiling face hangs over a park at night. a young couple walking past a fence learn on a railing and look up. the moon smiles. they embrace, and the moon's smile gets bigger. they then sit down on a bench by a tree. the moon's view is blocked, causing him to frown. in the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better."
2,1901,the martyred presidents,american,,,,https://en.wikipedia.org/wiki/the_martyred_presidents,"the film, just over a minute long, is composed of two shots. in the first, a girl sits at the base of an altar or tomb, her face hidden from the camera. at the center of the altar, a viewing portal displays the portraits of three u.s. presidents—abraham lincoln, james a. garfield, and william mckinley—each victims of assassination.\r\nin the second shot, which runs just over eight seconds long, an assassin kneels feet of lady justice.","1901 1901 1901 1901 1901 1901 1901 1901 1901 1901 the martyred presidents american american american american american the film, just over a minute long, is composed of two shots. in the first, a girl sits at the base of an altar or tomb, her face hidden from the camera. at the center of the altar, a viewing portal displays the portraits of three u.s. presidents—abraham lincoln, james a. garfield, and william mckinley—each victims of assassination.\r\nin the second shot, which runs just over eight seconds long, an assassin kneels feet of lady justice."
3,1901,"terrible teddy, the grizzly king",american,,,,"https://en.wikipedia.org/wiki/terrible_teddy,_the_grizzly_king","lasting just 61 seconds and consisting of two shots, the first shot is set in a wood during winter. the actor representing then vice-president theodore roosevelt enthusiastically hurries down a hillside towards a tree in the foreground. he falls once, but rights himself and cocks his rifle. two other men, bearing signs reading ""his photographer"" and ""his press agent""

## Removing Special Characters

In [17]:
df["doc"] = df["doc"].str.replace("[^a-z 0-9]+", "", regex=True)

In [18]:
pd.set_option('display.max_colwidth', None)
df[["doc"]].sample(10)

doc
15477                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             2008 2008 2008 2008 2008 mamma mia american american american directorphyllidalloyd directorphyllidalloyd directorphyllidalloyd actormerylstreep actoramandaseyfried actorpiercebros actorcolinfirth actorstellanskarsgrd actorchristinebaranski musical musical musical musical musical on the greek island of kalokairi 20yearold bridetobe sophie sheridan reveals to her bridesmaids that she has secretly invited three men to her wedding without telling her mother donna they are the men that her mothers diary reveals could have been her father irishamerican architect sam carmichael swedish adventurer and writer bill anderson and british banker harry bright she dreams of being given away by her father at her wedding and believes that after she spends time with them she will know which is her fathersophies mother donna who owns a villa is ecstatic to reunite with her former dynamos bandmates wisecracking author rosie mulligan and wealthy multiple divorce tanya cheshamleigh and reveals her bafflement at her daughters desire to get married donna shows off the villa to rosie and tanya the three men arrive and sophie smuggles them to their room she does not reveal that she believes one of them is her father but does explain that she and not her mother sent the invitations she begs them to hide so donna will be surprised by the old friends of whom she so often favorably speaks they overhear donna working and swear not to reveal sophies secretdonna spies them and is dumbfounded to find herself facing former lovers demanding they leave she confides in tanya and rosie that she truly does not know which of the three fathered sophie tanya and rosie rally her spirits by getting her to dance with an all female ensemble of staff and islanders sophie finds the men aboard bills yacht and they sail around kalokairi telling stories of donnas carefree youth sophie plans to tell her fianc sky about her ploy but loses her nerve sky and sophie sing to each other but sky is abducted for his bachelor partyat sophies bachelorette party donna tanya and rosie perform when sam bill and harry arrive sophie decides to talk with each of them alone while her girlfriends dance with the men sophie learns from bill that donna received the money for her villa from his greataunt sofia sophie guesses she must be sofias namesake she asks him to give her away and keep their secret until the wedding sophies happiness is shortlived as sam and harry each pull her aside to tell her they are her father and will give her away sophie overwhelmed by the consequences of raising the hopes of all three fathers faintsin the morning rosie and tanya assure donna they will take care of the men bill and harry intend to tell each other what they learned the previous night but rosie interrupts them donna confronts sophie believing sophie wants the wedding stopped sophie says

## Removing Stopwords

In [191]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords


In [192]:
stops = stopwords.words('english')
print(stops)

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [193]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess(sentence):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(lemma_words)

df['doc_clean'] = df['doc'].map(lambda s:preprocess(s)) 

In [195]:
df[["doc", 'doc_clean']].sample(10)

doc  \
1666                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 1934 the count of monte cristo american directorrowlandvlee actorrobertdonat actorelissalandi actorlouiscalhern drama adventure in 1815 a french merchant ship stops at the island of elba a letter from the exiled napoleon is given to the ships captain to deliver to a man in marseille before he dies of a sickness the captain entrusts the task to his first officer edmond dants donat however the city magistrate raymond de villefort jr calhern is tipped off by an informer the second officer danglars raymond walburn and has both men arrested after the exchangedants friend fernand mondego sidney blackmer accompanies him to the jail however he danglars and de villefort all stand to gain from keeping dants imprisoned mondego is in love with dants fiance mercedes landi danglars wants to be promoted captain in dants place and the man who accepted the letter turns out to be de villeforts father lawrence grant de villefort consigns dants without trial to a notorious prison the chteau dif on the false testimony of danglarswhen napoleon returns to france giving dants friends hope for his release de villefort signs a false statement that he was killed trying to escape which mondego shows to mercedes deceived she gives in to her mothers deathbed wish and marries mondegoeight years of solitary confinement follow for dants then one day the aged abb faria o p heggie a fellow prisoner breaks into his cell through a tunnel he has been digging the two join forces faria calculates it will take five more years to finish in the meantime he starts educating dantshowever as they near their goal a cavein fatally injures the old man before he dies he bequeaths a vast hidden treasure to his protg farias enemies had tortured and imprisoned him in an unsuccessful attempt to extract its location the body is sewn into a shroud but while the undertaker is away dants substitutes himself for the corpse undetected he is cast into the sea he frees himself and is picked up by a smuggling shipdants later follows the abbs directions and finds the treasure on the uninhabited island of monte cristo with a fortune at his command he sets in motion his plans for revenge to begin he arranges to have albert mercedes and mondegos son kidnapped and held for ransom dants rescues the younger man in order to gain entry into paris society using his purchased title of count of monte cristofirst to be brought to justice is mondego while the french ambassador to albania mondego gained renown for his bravery in an unsuccessful defense of ali pasha dants arranges a ball to honor his enemy then arranges to have him exposed publicly as the one who betrayed ali pasha to his death at the hands of the turks unaware of the counts role in his disgrace mondego goes to him for advice dants reveals his identity and they engage in a duel dants wins but spares mondego who 

## TF-IDF

In [196]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["doc_clean"])
column_names = vectorizer.get_feature_names_out()

df_tf_idf = pd.DataFrame(X.toarray(), columns=column_names)
df_tf_idf.shape

(34886, 251188)

## Cosine Similarity

In [197]:
from sklearn.metrics.pairwise import cosine_similarity

df_cos_sim = pd.DataFrame(cosine_similarity(df_tf_idf, dense_output=True))
df_cos_sim.shape

/opt/miniconda3/envs/ai/lib/python3.9/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


(34886, 34886)

In [198]:
df_cos_sim.to_parquet("../data/movie_cos_sim.parquet")

## Testing

In [203]:
pd.set_option('display.max_colwidth', 50)


In [204]:
query = 'titanic'

df[df["Title"].str.contains("titanic")]

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,doc,doc_clean
6008,1953,titanic,american,directorjeannegulesco,actorbarbarastanwyck actorcliftonwebb actorrob...,biography,https://en.wikipedia.org/wiki/titanic_(1953_film),"at the last minute, a wealthy american expatri...",1953 titanic american directorjeannegulesco ac...,1953 titan american directorjeannegulesco acto...
9758,1980,raise the titanic,american,directorjerryjameson,actorjasonrobards actoralecguinness actorannea...,action,https://en.wikipedia.org/wiki/raise_the_titani...,the film opens on the fictional island of svar...,1980 raise the titanic american directorjerryj...,1980 rais titan american directorjerryjameson ...
12857,1996,titanic,american,directorrobertlieberman,actorgeorgec.scott actorevamariesaint actorpet...,biography,https://en.wikipedia.org/wiki/titanic_(1996_tv...,titanic follows three main story threads.\r\ni...,1996 titanic american directorrobertlieberman ...,1996 titan american directorrobertlieberman ac...
13153,1997,titanic,american,directorjamescameron,actorleonardodicaprio actorkatewinslet actorbi...,"historical epic, disaster",https://en.wikipedia.org/wiki/titanic_(1997_film),"in 1996, treasure hunter brock lovett and his ...",1997 titanic american directorjamescameron act...,1997 titan american directorjamescameron actor...
16392,2012,titanic 3d,american,directorjamescameron,actorleonardodicaprio actorkatewinslet actorbi...,drama,https://en.wikipedia.org/wiki/titanic_(1997_film),"in 1996, treasure hunter brock lovett and his ...",2012 titanic 3d american directorjamescameron ...,2012 titan american directorjamescameron actor...


In [207]:
movie_index = 13153 # Titanic Movie

df_query = df_cos_sim[[movie_index]].sort_values(by=[movie_index]).tail(5)
df_query.shape

(34886, 1)

In [217]:
df_query

,13153
6275,0.398865
16004,0.421042
14599,0.472291
16392,0.992128
13153,1.000000


In [223]:
df[df.index == 6275]

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,doc,doc_clean
6275,1955,east of eden,american,directoreliakazan,actorjulieharris actorjamesdean actorraymondma...,drama,https://en.wikipedia.org/wiki/east_of_eden_(film),"the story is set during 1917 and 1918, leading...",1955 east of eden american directoreliakazan a...,1955 east eden american directoreliakazan acto...
